In [ ]:
#This notebook contains imutils to prepare data to Train Unet.
#Or other things too 

In [3]:
import tifffile as tiff
from natsort import natsorted
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cv2
from imutils.src import imfunctions #contours_length as imutils.contours_length
from natsort import natsorted
import shutil

In [ ]:
#First part: extract frames
frames2extract=np.arange(0,200000,17777)#(0,200000,10000)
input_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all/'
output_folder='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_test/'
try: os.mkdir(output_folder) # creates the subdirectory where it should be stored
except: print('Output Directory already exists, might overwrite')

for btf_file in natsorted(os.listdir(input_folder)):
    print(btf_file)
    if '2020-07' in btf_file:
        print(btf_file)
        btf_file_basename=os.path.splitext(os.path.basename(btf_file))[0]
        print(btf_file_basename)
        imfunctions.extract_frames(os.path.join(input_folder,btf_file), os.path.join(output_folder,btf_file_basename),frames2extract)
        print('end')

In [ ]:
#add zeros (just in case)
path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_test/'
for dirpath, _, files in os.walk(path):
    print(dirpath)
    imfunctions.add_zeros_to_filename(dirpath, 6)

In [ ]:
#rename to have unique names (prepend folder name into file name)
path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_test/'
for dirpath, _, files in os.walk(path):
    for f in files:
        os.rename(os.path.join(dirpath, f), os.path.join(dirpath, os.path.split(dirpath)[-1] + '_' + f))

In [ ]:
#copy all files from subfolders into a new single folder
path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_test/'
dst_path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_test_all/'

try: os.mkdir(dst_path) # creates the subdirectory where it should be stored
except: print('Output Directory already exists, might overwrite')

for directory, subdirectories, files in os.walk(path):
    for file in files:
        #print(file)
        shutil.copyfile(os.path.join(directory,file),os.path.join(dst_path,file))


In [8]:
#substract background if wanted

bg_img_filename='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/background/2020-07-01_13-42-05_chemotaxisl_worm1_bg-channel-0-_MMStack_Median_BG.tiff'
bg_img=tiff.imread(bg_img_filename)

path='/groups/zimmer/Ulises/code/unet-master/data/worm2/test2/'

#this code is flexible to files being in separate folders
for directory, subdirectories, files in os.walk(path):
    for file in files:
        filepath=os.path.join(directory,file)
        img=tiff.imread(filepath)
        inv_img=cv2.bitwise_not(img)
        new_img=cv2.subtract(inv_img,bg_img)
        tiff.imwrite(filepath,new_img)

In [ ]:
#Use draw_some_contours function to extract interesting contours
path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_training_all_image/'

for directory, subdirectories, files in os.walk(path):
    for file in files:
        filepath=os.path.join(directory,file)
        img=tiff.imread(filepath)
        #median Blur
        img=cv2.medianBlur(img,5)
       
        #apply threshold
        ret, new_img = cv2.threshold(img,20,255,cv2.THRESH_BINARY)
        worm_contour_img=imfunctions.draw_some_contours(new_img,contour_size=8000,tolerance=0.25, inner_contour_area_to_fill=300)
        
        #if you want to plot the different stages
        #plotting
#         fig, (ax1,ax2)=plt.subplots(figsize=(10,5), nrows=1, ncols=2, dpi=100)
#         im=ax1.imshow(img)
#         divider = make_axes_locatable(ax1)
#         cax = divider.append_axes('right', size='5%', pad=0.05)
#         fig.colorbar(im, cax=cax, orientation='vertical')
#         ax2.imshow(worm_contour_img)
#         [axi.set_axis_off() for axi in (ax1,ax2)]
        
        tiff.imwrite(filepath,worm_contour_img)

In [ ]:
#convert to stack if wanted to screen for masks that need improvement (eg remove bubble)
path='/groups/zimmer/Ulises/code/unet-master/data/new_training_all_image/'
o_path='/groups/zimmer/Ulises/code/unet-master/data/new_training_all_stack.tif'
imfunctions.images2stack(path, o_path)

In [7]:
#manually annotate frames to correct
path='/groups/zimmer/Ulises/code/unet-master/data/worm2_background_removed/test/'
idxs=[53,76,98]#[15,67,80,89,92,113,157,158,169]
#get their filename
for idx,file in enumerate(natsorted(os.listdir(path))):
    #add +1 if idxs are starting from 1
    if idx+1 in idxs:
        print(idx+1, file)

53 2020-07-01_14-41-11_chemotaxisl_worm2-channel-0-bigtiff_img071108.tif
76 2020-07-01_16-20-36_chemotaxisl_worm5-channel-0-bigtiff_img053331.tif
98 2020-07-01_17-32-12_control_worm4-channel-0-bigtiff_img017777.tif
